# Convert correction profiles from 20201012 30-zsteps into 50 steps

by Pu Zheng

2020.12.04

In [1]:
%run "E:\Users\puzheng\Documents\Startup_py3.py"
sys.path.append(r"E:\Users\puzheng\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *

print(os.getpid())

36400


In [4]:
source_folder = r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20201012-Corrections_3color'
target_folder = r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210203-Corrections_3color_35'
if not os.path.exists(target_folder):
    print(f"create folder: {target_folder}")
    os.makedirs(target_folder)
    
src_shape = [30,2048,2048]
tar_shape = [35,2048,2048]

create folder: \\10.245.74.158\Chromatin_NAS_0\Corrections\20210203-Corrections_3color_35


# illumination corr

In [5]:
from shutil import copyfile

file_prefix = 'illumination_correction_'
channels = ia._allowed_colors

for _ch in channels:
    _src_fl = os.path.join(source_folder, 
                           file_prefix+f"{_ch}_{src_shape[-2]}x{src_shape[-1]}.npy")
    _tar_fl = os.path.join(target_folder, 
                           file_prefix+f"{_ch}_{tar_shape[-2]}x{tar_shape[-1]}.npy")
    if os.path.isfile(_src_fl):
        print(_ch)
        copyfile(_src_fl, _tar_fl)

750
647
561
488
405


# chromatic abbrevation corr

In [6]:
# load chromatic abbrevation pf
file_prefix = 'chromatic_correction_'
ref_ch = '647'
chromatic_corr_channels = ['750', '561']
for _ch in chromatic_corr_channels:
    _basename = file_prefix+f"{_ch}_{ref_ch}_{src_shape[-2]}x{src_shape[-1]}.npy"
    _src_fl = os.path.join(source_folder, 
                           file_prefix+f"{_ch}_{ref_ch}_{src_shape[-3]}_{src_shape[-2]}_{src_shape[-1]}_const.pkl")
    _tar_fl = os.path.join(target_folder, 
                           file_prefix+f"{_ch}_{ref_ch}_{tar_shape[-3]}_{tar_shape[-2]}_{tar_shape[-1]}_const.pkl")
    
    if os.path.isfile(_src_fl):
        print(_ch)
        _info = pickle.load(open(_src_fl, 'rb'))
        # generate new chromatic const file
        new_info = {_k:_v for _k,_v in _info.items()}
        new_info['ref_center'] = (np.array(tar_shape)/2)
        # save new info
        pickle.dump(new_info, open(_tar_fl, 'wb'))
        print(_tar_fl)
        ## generate profiles
        # generate pixels
        _pixel_coords = np.indices(tar_shape)
        _pixel_coords = _pixel_coords.reshape(np.shape(_pixel_coords)[0], -1)
        _pixel_coords = _pixel_coords - (np.array(tar_shape)/2)[:, np.newaxis]
        from ImageAnalysis3.correction_tools.chromatic import generate_polynomial_data
        # generate profile
        _profile = []
        for _i, _order in enumerate(_info['fitting_orders']):
            _pX = generate_polynomial_data(_pixel_coords.transpose(), _order)
            _py = np.dot(_pX, _info['constants'][_i]).reshape(tar_shape)
            _profile.append(_py)
        _profile = np.array(_profile)
        # save profile
        np.save(_tar_fl.replace("_const.pkl", ""), _profile)

750
\\10.245.74.158\Chromatin_NAS_0\Corrections\20210203-Corrections_3color_35\chromatic_correction_750_647_35_2048_2048_const.pkl
561
\\10.245.74.158\Chromatin_NAS_0\Corrections\20210203-Corrections_3color_35\chromatic_correction_561_647_35_2048_2048_const.pkl


# bleedthrough corr

In [7]:
# load chromatic abbrevation pf
file_prefix = 'bleedthrough_correction_'
bleed_corr_channels = ['750', '647', '561']
#750_647_561_2048_2048
_basename = file_prefix
for _ch in bleed_corr_channels:
    _basename += f"{_ch}_"
_basename += f"{src_shape[-2]}_{src_shape[-1]}.npy"
_src_fl = os.path.join(source_folder, _basename)
_pf = np.load(_src_fl)

In [8]:
_tar_fl = os.path.join(target_folder, _basename)
np.save(_tar_fl.split('.npy')[0], _pf)

In [9]:
# generate a 2color bleedthrough profile

# load chromatic abbrevation pf
file_prefix = 'bleedthrough_correction_'
src_bleed_corr_channels = ['750', '647', '561']
#750_647_561_2048_2048
_src_basename = file_prefix
for _ch in src_bleed_corr_channels:
    _src_basename += f"{_ch}_"
_src_basename += f"{src_shape[-2]}_{src_shape[-1]}.npy"
_src_fl = os.path.join(source_folder, _src_basename)
_pf = np.load(_src_fl)


In [10]:
_pf = _pf.reshape(len(src_bleed_corr_channels), 
                  len(src_bleed_corr_channels), 
                  src_shape[-2], src_shape[-1])

tar_bleed_corr_channels = ['750', '647']

_tar_basename = file_prefix
for _ch in tar_bleed_corr_channels:
    _tar_basename += f"{_ch}_"
_tar_basename += f"{tar_shape[-2]}_{tar_shape[-1]}"
_tar_fl = os.path.join(target_folder, _tar_basename)

_tar_pf = _pf[:len(tar_bleed_corr_channels),:len(tar_bleed_corr_channels)]
_tar_pf = _tar_pf.reshape(len(tar_bleed_corr_channels)**2, tar_shape[-2], tar_shape[-1])

np.save(_tar_fl, _tar_pf)

In [11]:
_tar_fl

'\\\\10.245.74.158\\Chromatin_NAS_0\\Corrections\\20210203-Corrections_3color_35\\bleedthrough_correction_750_647_2048_2048'